In [1]:
import category_encoders as ce
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()

'/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/notebooks'

In [144]:
df = pd.read_csv('../data/03_primary/df_train.csv')

In [20]:
df

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,1,Android,Chrome,Germany,Dusseldorf,google,organic,NaN,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,0,Windows,Firefox,United States,(not set),google,organic,NaN,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,0,Web,Chrome,Netherlands,Amsterdam,shop.googlemerchandisestore.com,referral,NaN,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,1,Web,Chrome,Australia,Perth,(direct),(none),NaN,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,1,iOS,Safari,United States,Charlotte,<Other>,referral,NaN,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,1,Android,Chrome,Malaysia,Kuala Lumpur,<Other>,<Other>,NaN,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,0,Web,Chrome,United States,(not set),<Other>,referral,NaN,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,0,Web,Safari,United States,(not set),google,organic,NaN,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,0,Macintosh,Chrome,United States,(not set),(direct),(none),NaN,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,1,Web,<Other>,United States,Columbus,<Other>,referral,NaN,0,0,12,5,3,NaN,0,train


In [21]:
for i in df.columns:
    print(i)

i_full_visitor_id
i_visit_start_time
i_date
c_device_category
c_is_mobile
c_operating_system
c_browser
c_country
c_city
c_traffic_source
c_traffic_medium
c_traffic_campaign
c_is_first_visit
n_product_pages_viewed
n_total_hits
n_total_pageviews
n_total_visits
n_total_time_on_site
y_added_to_cart
i_subset


In [22]:
encodings = {
    'bool': ['c_is_mobile', 'c_is_first_visit'],
    'ohe': ['c_device_category', 'c_operating_system', 'c_browser', 'c_traffic_source', 'c_traffic_medium'],
    'int': ['c_country', 'c_city', 'c_traffic_campaign', 'c_weekday', 'c_visit_start_hour']
}

In [23]:
binary_encoder = ce.BinaryEncoder(cols=encodings['bool'])

In [24]:
binary_encoder.fit(df)
binary_encoder.transform(df)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile_0,c_is_mobile_1,c_operating_system,c_browser,c_country,c_city,...,c_traffic_campaign,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,0,1,Android,Chrome,Germany,Dusseldorf,...,NaN,0,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,1,0,Windows,Firefox,United States,(not set),...,NaN,0,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,1,0,Web,Chrome,Netherlands,Amsterdam,...,NaN,0,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,0,1,Web,Chrome,Australia,Perth,...,NaN,0,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,0,1,iOS,Safari,United States,Charlotte,...,NaN,0,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,0,1,Android,Chrome,Malaysia,Kuala Lumpur,...,NaN,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,1,0,Web,Chrome,United States,(not set),...,NaN,0,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,1,0,Web,Safari,United States,(not set),...,NaN,0,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,1,0,Macintosh,Chrome,United States,(not set),...,NaN,1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,0,1,Web,<Other>,United States,Columbus,...,NaN,1,0,0,12,5,3,NaN,0,train


In [25]:
df_cp = df.copy()
df_cp.iloc[39, 12] = None
df_cp.iloc[:, 12]

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     0.0
6     1.0
7     1.0
8     0.0
9     0.0
10    1.0
11    1.0
12    0.0
13    1.0
14    1.0
15    0.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    0.0
29    1.0
30    1.0
31    1.0
32    1.0
33    1.0
34    1.0
35    1.0
36    1.0
37    1.0
38    1.0
39    NaN
Name: c_is_first_visit, dtype: float64

In [27]:
binary_encoder.fit(df)
binary_encoder.transform(df_cp)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile_0,c_is_mobile_1,c_operating_system,c_browser,c_country,c_city,...,c_traffic_campaign,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,0,1,Android,Chrome,Germany,Dusseldorf,...,NaN,0,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,1,0,Windows,Firefox,United States,(not set),...,NaN,0,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,1,0,Web,Chrome,Netherlands,Amsterdam,...,NaN,0,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,0,1,Web,Chrome,Australia,Perth,...,NaN,0,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,0,1,iOS,Safari,United States,Charlotte,...,NaN,0,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,0,1,Android,Chrome,Malaysia,Kuala Lumpur,...,NaN,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,1,0,Web,Chrome,United States,(not set),...,NaN,0,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,1,0,Web,Safari,United States,(not set),...,NaN,0,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,1,0,Macintosh,Chrome,United States,(not set),...,NaN,1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,0,1,Web,<Other>,United States,Columbus,...,NaN,1,0,0,12,5,3,NaN,0,train


In [28]:
binary_encoder.fit(df_cp)
binary_encoder.transform(df_cp)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile_0,c_is_mobile_1,c_operating_system,c_browser,c_country,c_city,...,c_traffic_campaign,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,0,1,Android,Chrome,Germany,Dusseldorf,...,NaN,0,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,1,0,Windows,Firefox,United States,(not set),...,NaN,0,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,1,0,Web,Chrome,Netherlands,Amsterdam,...,NaN,0,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,0,1,Web,Chrome,Australia,Perth,...,NaN,0,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,0,1,iOS,Safari,United States,Charlotte,...,NaN,0,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,0,1,Android,Chrome,Malaysia,Kuala Lumpur,...,NaN,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,1,0,Web,Chrome,United States,(not set),...,NaN,0,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,1,0,Web,Safari,United States,(not set),...,NaN,0,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,1,0,Macintosh,Chrome,United States,(not set),...,NaN,1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,0,1,Web,<Other>,United States,Columbus,...,NaN,1,0,0,12,5,3,NaN,0,train


In [29]:
onehot_encoder = ce.OneHotEncoder(cols=encodings['ohe'], use_cat_names=True)

In [31]:
onehot_encoder.fit(df)
onehot_encoder.transform(df)

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future ver

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_mobile,c_device_category_desktop,c_is_mobile,c_operating_system_Android,c_operating_system_Windows,c_operating_system_Web,c_operating_system_iOS,...,c_traffic_medium_(data deleted),c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,1,0,1,1,0,0,0,...,0,NaN,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,0,1,0,0,1,0,0,...,0,NaN,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,0,1,0,0,0,1,0,...,0,NaN,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,1,0,1,0,0,1,0,...,0,NaN,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,1,0,1,0,0,0,1,...,0,NaN,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,1,0,1,1,0,0,0,...,0,NaN,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,0,1,0,0,0,1,0,...,0,NaN,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,0,1,0,0,0,1,0,...,0,NaN,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,0,1,0,0,0,0,0,...,0,NaN,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,1,0,1,0,0,1,0,...,0,NaN,0,0,12,5,3,NaN,0,train


In [32]:
df_cp = df.copy()
df_cp.iloc[39, 3] = None
df_cp.iloc[:, 3]

0      mobile
1     desktop
2     desktop
3      mobile
4      mobile
5      mobile
6     desktop
7     desktop
8     desktop
9      mobile
10    desktop
11     mobile
12     mobile
13    desktop
14    desktop
15     mobile
16    desktop
17     mobile
18    desktop
19     mobile
20    desktop
21    desktop
22    desktop
23     mobile
24    desktop
25    desktop
26     mobile
27    desktop
28    desktop
29    desktop
30     mobile
31    desktop
32    desktop
33    desktop
34    desktop
35    desktop
36    desktop
37    desktop
38    desktop
39       None
Name: c_device_category, dtype: object

In [33]:
onehot_encoder.fit(df)
onehot_encoder.transform(df_cp)

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future ver

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_mobile,c_device_category_desktop,c_is_mobile,c_operating_system_Android,c_operating_system_Windows,c_operating_system_Web,c_operating_system_iOS,...,c_traffic_medium_(data deleted),c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,1,0,1,1,0,0,0,...,0,NaN,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,0,1,0,0,1,0,0,...,0,NaN,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,0,1,0,0,0,1,0,...,0,NaN,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,1,0,1,0,0,1,0,...,0,NaN,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,1,0,1,0,0,0,1,...,0,NaN,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,1,0,1,1,0,0,0,...,0,NaN,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,0,1,0,0,0,1,0,...,0,NaN,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,0,1,0,0,0,1,0,...,0,NaN,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,0,1,0,0,0,0,0,...,0,NaN,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,1,0,1,0,0,1,0,...,0,NaN,0,0,12,5,3,NaN,0,train


In [34]:
onehot_encoder.fit(df_cp)
onehot_encoder.transform(df_cp)

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future ver

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_mobile,c_device_category_desktop,c_device_category_nan,c_is_mobile,c_operating_system_Android,c_operating_system_Windows,c_operating_system_Web,...,c_traffic_medium_(data deleted),c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,1,0,0,1,1,0,0,...,0,NaN,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,0,1,0,0,0,1,0,...,0,NaN,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,0,1,0,0,0,0,1,...,0,NaN,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,1,0,0,1,0,0,1,...,0,NaN,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,1,0,0,1,0,0,0,...,0,NaN,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,1,0,0,1,1,0,0,...,0,NaN,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,0,1,0,0,0,0,1,...,0,NaN,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,0,1,0,0,0,0,1,...,0,NaN,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,0,1,0,0,0,0,0,...,0,NaN,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,1,0,0,1,0,0,1,...,0,NaN,0,0,12,5,3,NaN,0,train


In [35]:
ordinal_encoder = ce.OrdinalEncoder(cols=encodings['int'])

In [36]:
ordinal_encoder.fit(df)
ordinal_encoder.transform(df)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,1,Android,Chrome,1,1,google,organic,1,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,0,Windows,Firefox,2,2,google,organic,1,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,0,Web,Chrome,3,3,shop.googlemerchandisestore.com,referral,1,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,1,Web,Chrome,4,4,(direct),(none),1,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,1,iOS,Safari,2,5,<Other>,referral,1,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,1,Android,Chrome,5,6,<Other>,<Other>,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,0,Web,Chrome,2,2,<Other>,referral,1,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,0,Web,Safari,2,2,google,organic,1,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,0,Macintosh,Chrome,2,2,(direct),(none),1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,1,Web,<Other>,2,7,<Other>,referral,1,0,0,12,5,3,NaN,0,train


In [41]:
df_cp = df.copy()
df_cp.iloc[39, 7] = None
df_cp.iloc[:, 7]

0           Germany
1     United States
2       Netherlands
3         Australia
4     United States
5          Malaysia
6     United States
7     United States
8     United States
9     United States
10    United States
11            China
12    United States
13      New Zealand
14           Canada
15           Canada
16      Netherlands
17            India
18            India
19           Canada
20    United States
21    United States
22    United States
23          Morocco
24        Singapore
25    United States
26    United States
27        Singapore
28         Colombia
29    United States
30    United States
31            India
32    United States
33    United States
34            India
35           Poland
36    United States
37    United States
38    United States
39             None
Name: c_country, dtype: object

In [42]:
ordinal_encoder.fit(df)
ordinal_encoder.transform(df_cp)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,1,Android,Chrome,1,1,google,organic,1,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,0,Windows,Firefox,2,2,google,organic,1,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,0,Web,Chrome,3,3,shop.googlemerchandisestore.com,referral,1,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,1,Web,Chrome,4,4,(direct),(none),1,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,1,iOS,Safari,2,5,<Other>,referral,1,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,1,Android,Chrome,5,6,<Other>,<Other>,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,0,Web,Chrome,2,2,<Other>,referral,1,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,0,Web,Safari,2,2,google,organic,1,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,0,Macintosh,Chrome,2,2,(direct),(none),1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,1,Web,<Other>,2,7,<Other>,referral,1,0,0,12,5,3,NaN,0,train


In [43]:
ordinal_encoder.fit(df_cp)
ordinal_encoder.transform(df_cp)

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,mobile,1,Android,Chrome,1,1,google,organic,1,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,desktop,0,Windows,Firefox,2,2,google,organic,1,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,desktop,0,Web,Chrome,3,3,shop.googlemerchandisestore.com,referral,1,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,mobile,1,Web,Chrome,4,4,(direct),(none),1,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,mobile,1,iOS,Safari,2,5,<Other>,referral,1,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,mobile,1,Android,Chrome,5,6,<Other>,<Other>,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,desktop,0,Web,Chrome,2,2,<Other>,referral,1,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,desktop,0,Web,Safari,2,2,google,organic,1,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,desktop,0,Macintosh,Chrome,2,2,(direct),(none),1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,mobile,1,Web,<Other>,2,7,<Other>,referral,1,0,0,12,5,3,NaN,0,train


In [47]:
df_cp2 = df.copy()

In [48]:
binary_encoder = ce.BinaryEncoder(cols=encodings['bool'])
onehot_encoder = ce.OneHotEncoder(cols=encodings['ohe'], use_cat_names=True)
ordinal_encoder = ce.OrdinalEncoder(cols=encodings['int'])

In [49]:
binary_encoder.fit(df_cp2)
df_cp2 = binary_encoder.transform(df_cp2)
onehot_encoder.fit(df_cp2)
df_cp2 = onehot_encoder.transform(df_cp2)
ordinal_encoder.fit(df_cp2)
df_cp2 = ordinal_encoder.transform(df_cp2)
df_cp2

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future ver

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_mobile,c_device_category_desktop,c_is_mobile_0,c_is_mobile_1,c_operating_system_Android,c_operating_system_Windows,c_operating_system_Web,...,c_traffic_campaign,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset
0,4.443510e+07,1.611645e+15,20210126,1,0,0,1,1,0,0,...,1,0,1,0,3,1,1,NaN,0,train
1,2.572363e+06,1.611844e+15,20210128,0,1,1,0,0,1,0,...,1,0,1,0,4,2,1,45.0,0,train
2,7.820448e+06,1.611628e+15,20210126,0,1,1,0,0,0,1,...,1,0,1,0,6,2,1,4524.0,0,train
3,8.955082e+07,1.611873e+15,20210128,1,0,0,1,0,0,1,...,1,0,1,0,6,2,1,5271.0,0,train
4,5.293865e+09,1.611694e+15,20210126,1,0,0,1,0,0,0,...,1,0,1,1,6,2,1,4806.0,0,train
5,6.321521e+07,1.611631e+15,20210126,1,0,0,1,1,0,0,...,1,1,0,0,5,2,1,2061.0,0,train
6,6.349855e+07,1.611814e+15,20210128,0,1,1,0,0,0,1,...,1,0,1,0,6,4,1,NaN,0,train
7,8.594325e+06,1.611769e+15,20210127,0,1,1,0,0,0,1,...,1,0,1,0,9,4,1,40407.0,0,train
8,3.604692e+06,NaN,20210127,0,1,1,0,0,0,0,...,1,1,0,1,39,10,1,50874.0,1,train
9,1.297418e+06,1.611650e+15,20210126,1,0,0,1,0,0,1,...,1,1,0,0,12,5,3,NaN,0,train


---

In [51]:
df.dtypes

i_full_visitor_id         float64
i_visit_start_time        float64
i_date                      int64
c_device_category          object
c_is_mobile                 int64
c_operating_system         object
c_browser                  object
c_country                  object
c_city                     object
c_traffic_source           object
c_traffic_medium           object
c_traffic_campaign        float64
c_is_first_visit            int64
n_product_pages_viewed      int64
n_total_hits                int64
n_total_pageviews           int64
n_total_visits              int64
n_total_time_on_site      float64
y_added_to_cart             int64
i_subset                   object
dtype: object

In [63]:
pd.to_datetime(df['i_visit_start_time'], unit='us')

0    2021-01-26 07:11:18.583576064
1    2021-01-28 14:21:02.956374016
2    2021-01-26 02:20:58.126790912
3    2021-01-28 22:25:40.729475072
4    2021-01-26 20:39:19.514567936
5    2021-01-26 03:10:39.811727104
6    2021-01-28 06:09:34.021603072
7    2021-01-27 17:44:32.084555008
8                              NaT
9    2021-01-26 08:30:15.913019904
10   2021-01-26 10:40:25.201136128
11   2021-01-26 02:24:30.919459072
12   2021-01-26 20:56:42.563680000
13   2021-01-27 21:14:04.269050880
14   2021-01-27 21:01:00.664721920
15   2021-01-28 03:56:00.729658880
16   2021-01-26 03:01:40.601113088
17   2021-01-27 03:00:21.920994048
18   2021-01-26 20:01:09.034866944
19   2021-01-27 04:11:38.541039104
20   2021-01-29 10:30:54.912690944
21   2021-01-29 13:08:29.564285952
22   2021-01-29 23:42:44.265383936
23   2021-01-29 03:25:04.055139072
24   2021-01-29 07:38:03.648776960
25   2021-01-29 04:13:01.463984128
26   2021-01-29 09:10:40.285879040
27   2021-01-29 04:02:26.127554048
28   2021-01-29 20:4

In [60]:
pd.to_datetime(df['i_visit_start_time'], unit='us').dt.weekday

0     1.0
1     3.0
2     1.0
3     3.0
4     1.0
5     1.0
6     3.0
7     2.0
8     NaN
9     1.0
10    1.0
11    1.0
12    1.0
13    2.0
14    2.0
15    3.0
16    1.0
17    2.0
18    1.0
19    2.0
20    4.0
21    4.0
22    4.0
23    4.0
24    4.0
25    4.0
26    4.0
27    4.0
28    4.0
29    4.0
30    5.0
31    5.0
32    5.0
33    5.0
34    5.0
35    5.0
36    5.0
37    5.0
38    5.0
39    5.0
Name: i_visit_start_time, dtype: float64

In [62]:
pd.to_datetime(df['i_visit_start_time'], unit='us').dt.hour

0      7.0
1     14.0
2      2.0
3     22.0
4     20.0
5      3.0
6      6.0
7     17.0
8      NaN
9      8.0
10    10.0
11     2.0
12    20.0
13    21.0
14    21.0
15     3.0
16     3.0
17     3.0
18    20.0
19     4.0
20    10.0
21    13.0
22    23.0
23     3.0
24     7.0
25     4.0
26     9.0
27     4.0
28    20.0
29     7.0
30     2.0
31     3.0
32     7.0
33    12.0
34     0.0
35    22.0
36    22.0
37    14.0
38    16.0
39    16.0
Name: i_visit_start_time, dtype: float64

In [64]:
df_cp2['c_weekday'] = pd.to_datetime(df['i_visit_start_time'], unit='us').dt.weekday
df_cp2['c_visit_start_hour'] = pd.to_datetime(df['i_visit_start_time'], unit='us').dt.hour
df_cp2

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_mobile,c_device_category_desktop,c_is_mobile_0,c_is_mobile_1,c_operating_system_Android,c_operating_system_Windows,c_operating_system_Web,...,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,i_subset,c_weekday,c_visit_start_hour
0,4.443510e+07,1.611645e+15,20210126,1,0,0,1,1,0,0,...,1,0,3,1,1,NaN,0,train,1.0,7.0
1,2.572363e+06,1.611844e+15,20210128,0,1,1,0,0,1,0,...,1,0,4,2,1,45.0,0,train,3.0,14.0
2,7.820448e+06,1.611628e+15,20210126,0,1,1,0,0,0,1,...,1,0,6,2,1,4524.0,0,train,1.0,2.0
3,8.955082e+07,1.611873e+15,20210128,1,0,0,1,0,0,1,...,1,0,6,2,1,5271.0,0,train,3.0,22.0
4,5.293865e+09,1.611694e+15,20210126,1,0,0,1,0,0,0,...,1,1,6,2,1,4806.0,0,train,1.0,20.0
5,6.321521e+07,1.611631e+15,20210126,1,0,0,1,1,0,0,...,0,0,5,2,1,2061.0,0,train,1.0,3.0
6,6.349855e+07,1.611814e+15,20210128,0,1,1,0,0,0,1,...,1,0,6,4,1,NaN,0,train,3.0,6.0
7,8.594325e+06,1.611769e+15,20210127,0,1,1,0,0,0,1,...,1,0,9,4,1,40407.0,0,train,2.0,17.0
8,3.604692e+06,NaN,20210127,0,1,1,0,0,0,0,...,0,1,39,10,1,50874.0,1,train,NaN,NaN
9,1.297418e+06,1.611650e+15,20210126,1,0,0,1,0,0,1,...,0,0,12,5,3,NaN,0,train,1.0,8.0


---

In [72]:
columns_to_encode = [item for sublist in list(encodings.values()) for item in sublist]
columns_to_encode

['c_is_mobile',
 'c_is_first_visit',
 'c_device_category',
 'c_operating_system',
 'c_browser',
 'c_traffic_source',
 'c_traffic_medium',
 'c_country',
 'c_city',
 'c_traffic_campaign']

In [71]:
df.columns

Index(['i_full_visitor_id', 'i_visit_start_time', 'i_date',
       'c_device_category', 'c_is_mobile', 'c_operating_system', 'c_browser',
       'c_country', 'c_city', 'c_traffic_source', 'c_traffic_medium',
       'c_traffic_campaign', 'c_is_first_visit', 'n_product_pages_viewed',
       'n_total_hits', 'n_total_pageviews', 'n_total_visits',
       'n_total_time_on_site', 'y_added_to_cart', 'i_subset'],
      dtype='object')

In [75]:
[item in df.columns for item in columns_to_encode]

[True, True, True, True, True, True, True, True, True, True]

In [76]:
all(item in df.columns for item in columns_to_encode)

True

In [77]:
[item in df.columns for item in columns_to_encode + ['added_col']]

[True, True, True, True, True, True, True, True, True, True, False]

In [78]:
all(item in df.columns for item in columns_to_encode + ['added_col'])

False

---

In [122]:
import re
from sklearn.impute import SimpleImputer

In [162]:
df = pd.read_csv('../data/03_primary/df_train.csv')

In [163]:
df

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
0,1.772899e+07,1.611713e+15,20210127,desktop,0,Web,Chrome,India,Jaipur,<Other>,<Other>,NaN,1,1,61,19,1,34848.0,0
1,3.431204e+07,1.611721e+15,20210127,desktop,0,Macintosh,Chrome,United States,New York,(direct),(none),NaN,1,0,5,2,1,1125.0,0
2,6.149654e+06,1.611737e+15,20210127,mobile,1,iOS,Safari,China,Chengdu,(direct),(none),NaN,0,0,318,87,5,479071.0,0
3,5.319757e+07,1.611623e+15,20210126,desktop,0,Web,Edge,Singapore,Singapore,shop.googlemerchandisestore.com,referral,NaN,1,0,3,1,1,NaN,0
4,3.294631e+07,1.611649e+15,20210126,desktop,0,Web,Chrome,United States,New York,google,organic,NaN,0,1,27,9,1,24954.0,0
5,4.367234e+06,1.611740e+15,20210127,mobile,1,Android,Chrome,Netherlands,The Hague,google,organic,NaN,1,1,6,2,1,1459.0,0
6,5.588531e+07,1.611681e+15,20210126,desktop,0,Web,Chrome,India,(not set),(direct),(none),NaN,1,0,6,2,1,12633.0,0
7,5.331924e+07,1.611670e+15,20210126,desktop,0,Macintosh,Chrome,Philippines,(not set),shop.googlemerchandisestore.com,referral,NaN,0,0,5,2,1,1420.0,0
8,8.550651e+07,1.611768e+15,20210127,desktop,0,Web,Edge,Turkey,Istanbul,<Other>,<Other>,NaN,1,0,4,2,1,32.0,0
9,3.268039e+07,1.611741e+15,20210127,desktop,0,Web,Chrome,India,Gurgaon,google,organic,NaN,1,1,7,2,1,11091.0,0


In [164]:
df.isnull().sum()

i_full_visitor_id          0
i_visit_start_time         1
i_date                     0
c_device_category          0
c_is_mobile                0
c_operating_system         0
c_browser                  0
c_country                  0
c_city                     0
c_traffic_source           0
c_traffic_medium           0
c_traffic_campaign        20
c_is_first_visit           0
n_product_pages_viewed     0
n_total_hits               0
n_total_pageviews          0
n_total_visits             0
n_total_time_on_site       3
y_added_to_cart            0
dtype: int64

In [165]:
df.dtypes

i_full_visitor_id         float64
i_visit_start_time        float64
i_date                      int64
c_device_category          object
c_is_mobile                 int64
c_operating_system         object
c_browser                  object
c_country                  object
c_city                     object
c_traffic_source           object
c_traffic_medium           object
c_traffic_campaign        float64
c_is_first_visit            int64
n_product_pages_viewed      int64
n_total_hits                int64
n_total_pageviews           int64
n_total_visits              int64
n_total_time_on_site      float64
y_added_to_cart             int64
dtype: object

In [166]:
num_cols = [item for item in df.columns if re.compile('^n_').match(item)]
cat_cols = [item for item in df.columns if re.compile('^c_').match(item)]

In [167]:
df[num_cols] = df[num_cols].astype(float)
df[cat_cols] = np.where(pd.isnull(df[cat_cols]), df[cat_cols], df[cat_cols].astype(str))

In [168]:
df.dtypes

i_full_visitor_id         float64
i_visit_start_time        float64
i_date                      int64
c_device_category          object
c_is_mobile                object
c_operating_system         object
c_browser                  object
c_country                  object
c_city                     object
c_traffic_source           object
c_traffic_medium           object
c_traffic_campaign         object
c_is_first_visit           object
n_product_pages_viewed    float64
n_total_hits              float64
n_total_pageviews         float64
n_total_visits            float64
n_total_time_on_site      float64
y_added_to_cart             int64
dtype: object

In [169]:
df

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
0,1.772899e+07,1.611713e+15,20210127,desktop,0,Web,Chrome,India,Jaipur,<Other>,<Other>,NaN,1,1.0,61.0,19.0,1.0,34848.0,0
1,3.431204e+07,1.611721e+15,20210127,desktop,0,Macintosh,Chrome,United States,New York,(direct),(none),NaN,1,0.0,5.0,2.0,1.0,1125.0,0
2,6.149654e+06,1.611737e+15,20210127,mobile,1,iOS,Safari,China,Chengdu,(direct),(none),NaN,0,0.0,318.0,87.0,5.0,479071.0,0
3,5.319757e+07,1.611623e+15,20210126,desktop,0,Web,Edge,Singapore,Singapore,shop.googlemerchandisestore.com,referral,NaN,1,0.0,3.0,1.0,1.0,NaN,0
4,3.294631e+07,1.611649e+15,20210126,desktop,0,Web,Chrome,United States,New York,google,organic,NaN,0,1.0,27.0,9.0,1.0,24954.0,0
5,4.367234e+06,1.611740e+15,20210127,mobile,1,Android,Chrome,Netherlands,The Hague,google,organic,NaN,1,1.0,6.0,2.0,1.0,1459.0,0
6,5.588531e+07,1.611681e+15,20210126,desktop,0,Web,Chrome,India,(not set),(direct),(none),NaN,1,0.0,6.0,2.0,1.0,12633.0,0
7,5.331924e+07,1.611670e+15,20210126,desktop,0,Macintosh,Chrome,Philippines,(not set),shop.googlemerchandisestore.com,referral,NaN,0,0.0,5.0,2.0,1.0,1420.0,0
8,8.550651e+07,1.611768e+15,20210127,desktop,0,Web,Edge,Turkey,Istanbul,<Other>,<Other>,NaN,1,0.0,4.0,2.0,1.0,32.0,0
9,3.268039e+07,1.611741e+15,20210127,desktop,0,Web,Chrome,India,Gurgaon,google,organic,NaN,1,1.0,7.0,2.0,1.0,11091.0,0


In [170]:
imputation_strategies = {
    'mean': ['n_total_time_on_site'],
    'zero': [],
    'mostfreq': [],
    # 'unknown': ['c_traffic_campaign', 'c_weekday', 'c_visit_start_hour'],
    'unknown': ['c_traffic_campaign'],
}

In [171]:
df_cp = df.copy()

In [172]:
if len(imputation_strategies['mean']) > 0:
    num_mean_imputer = SimpleImputer(strategy='mean')
    num_mean_imputer.fit(df[imputation_strategies['mean']])
else:
    num_mean_imputer = None

if len(imputation_strategies['zero']) > 0:
    num_zero_imputer = SimpleImputer(strategy='constant', fill_value=0.)
    num_zero_imputer.fit(df[imputation_strategies['zero']])
else:
    num_zero_imputer = None

if len(imputation_strategies['mostfreq']) > 0:
    cat_mostfreq_imputer = SimpleImputer(strategy='most_frequent')
    cat_mostfreq_imputer.fit(df[imputation_strategies['mostfreq']])
else:
    cat_mostfreq_imputer = None

if len(imputation_strategies['unknown']) > 0:
    cat_unknown_imputer = SimpleImputer(strategy='constant', fill_value='UNKNOWN')
    cat_unknown_imputer.fit(df[imputation_strategies['unknown']])
else:
    cat_unknown_imputer = None

In [173]:
if num_mean_imputer is not None:
    df_cp[num_mean_imputer.feature_names_in_] = num_mean_imputer.transform(df_cp[num_mean_imputer.feature_names_in_])

if num_zero_imputer is not None:
    df_cp[num_zero_imputer.feature_names_in_] = num_zero_imputer.transform(df_cp[num_zero_imputer.feature_names_in_])

if cat_mostfreq_imputer is not None:
    df_cp[cat_mostfreq_imputer.feature_names_in_] = cat_mostfreq_imputer.transform(df_cp[cat_mostfreq_imputer.feature_names_in_])

if cat_unknown_imputer is not None:
    df_cp[cat_unknown_imputer.feature_names_in_] = cat_unknown_imputer.transform(df_cp[cat_unknown_imputer.feature_names_in_])

In [174]:
df_cp.isnull().sum()

i_full_visitor_id         0
i_visit_start_time        1
i_date                    0
c_device_category         0
c_is_mobile               0
c_operating_system        0
c_browser                 0
c_country                 0
c_city                    0
c_traffic_source          0
c_traffic_medium          0
c_traffic_campaign        0
c_is_first_visit          0
n_product_pages_viewed    0
n_total_hits              0
n_total_pageviews         0
n_total_visits            0
n_total_time_on_site      0
y_added_to_cart           0
dtype: int64

In [175]:
df_cp

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
0,1.772899e+07,1.611713e+15,20210127,desktop,0,Web,Chrome,India,Jaipur,<Other>,<Other>,UNKNOWN,1,1.0,61.0,19.0,1.0,34848.000000,0
1,3.431204e+07,1.611721e+15,20210127,desktop,0,Macintosh,Chrome,United States,New York,(direct),(none),UNKNOWN,1,0.0,5.0,2.0,1.0,1125.000000,0
2,6.149654e+06,1.611737e+15,20210127,mobile,1,iOS,Safari,China,Chengdu,(direct),(none),UNKNOWN,0,0.0,318.0,87.0,5.0,479071.000000,0
3,5.319757e+07,1.611623e+15,20210126,desktop,0,Web,Edge,Singapore,Singapore,shop.googlemerchandisestore.com,referral,UNKNOWN,1,0.0,3.0,1.0,1.0,48837.411765,0
4,3.294631e+07,1.611649e+15,20210126,desktop,0,Web,Chrome,United States,New York,google,organic,UNKNOWN,0,1.0,27.0,9.0,1.0,24954.000000,0
5,4.367234e+06,1.611740e+15,20210127,mobile,1,Android,Chrome,Netherlands,The Hague,google,organic,UNKNOWN,1,1.0,6.0,2.0,1.0,1459.000000,0
6,5.588531e+07,1.611681e+15,20210126,desktop,0,Web,Chrome,India,(not set),(direct),(none),UNKNOWN,1,0.0,6.0,2.0,1.0,12633.000000,0
7,5.331924e+07,1.611670e+15,20210126,desktop,0,Macintosh,Chrome,Philippines,(not set),shop.googlemerchandisestore.com,referral,UNKNOWN,0,0.0,5.0,2.0,1.0,1420.000000,0
8,8.550651e+07,1.611768e+15,20210127,desktop,0,Web,Edge,Turkey,Istanbul,<Other>,<Other>,UNKNOWN,1,0.0,4.0,2.0,1.0,32.000000,0
9,3.268039e+07,1.611741e+15,20210127,desktop,0,Web,Chrome,India,Gurgaon,google,organic,UNKNOWN,1,1.0,7.0,2.0,1.0,11091.000000,0


In [176]:
df_test = df = pd.read_csv('../data/03_primary/df_test.csv')

In [177]:
df_cp = df_test

In [178]:
df_cp

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
0,5.116313e+07,1612045864216573,20210130,desktop,0,Web,Chrome,Russia,Moscow,shop.googlemerchandisestore.com,referral,NaN,1,0,8,3,1,20618.0,0
1,8.897099e+07,1612034970969348,20210130,desktop,0,Web,Chrome,United States,Salt Lake City,google,organic,NaN,1,0,4,2,1,26.0,0
2,5.210404e+07,1611968241031361,20210130,tablet,0,Web,Safari,United States,(not set),google,organic,NaN,1,0,4,2,1,12.0,0
3,7.754877e+07,1612040704762043,20210130,desktop,0,Macintosh,Chrome,United States,New Orleans,(direct),(none),NaN,1,0,6,2,1,5019.0,0
4,6.822834e+06,1611998301676244,20210130,mobile,1,Web,Safari,Canada,Brampton,(direct),(none),NaN,1,0,7,2,2,82.0,0
5,3.303965e+06,1611983150703412,20210130,mobile,1,iOS,Safari,United States,Los Angeles,<Other>,referral,NaN,1,0,4,2,1,24.0,0
6,5.390972e+07,1611972983188588,20210130,desktop,0,Web,Chrome,Cyprus,(not set),(direct),(none),NaN,1,0,4,2,1,44.0,0
7,5.997183e+06,1612004659459301,20210130,mobile,1,Web,Chrome,United States,(not set),<Other>,referral,NaN,0,0,3,2,1,NaN,0
8,3.279238e+07,1612036389028482,20210130,desktop,0,Windows,Chrome,Turkey,Adana,<Other>,referral,NaN,0,0,24,6,3,79106.0,0
9,7.534357e+06,1612050034319156,20210130,desktop,0,Web,Chrome,Canada,Calgary,(direct),(none),NaN,0,1,4,2,1,147.0,0


In [179]:
if num_mean_imputer is not None:
    df_cp[num_mean_imputer.feature_names_in_] = num_mean_imputer.transform(df_cp[num_mean_imputer.feature_names_in_])

if num_zero_imputer is not None:
    df_cp[num_zero_imputer.feature_names_in_] = num_zero_imputer.transform(df_cp[num_zero_imputer.feature_names_in_])

if cat_mostfreq_imputer is not None:
    df_cp[cat_mostfreq_imputer.feature_names_in_] = cat_mostfreq_imputer.transform(df_cp[cat_mostfreq_imputer.feature_names_in_])

if cat_unknown_imputer is not None:
    df_cp[cat_unknown_imputer.feature_names_in_] = cat_unknown_imputer.transform(df_cp[cat_unknown_imputer.feature_names_in_])

In [180]:
df_cp

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
0,5.116313e+07,1612045864216573,20210130,desktop,0,Web,Chrome,Russia,Moscow,shop.googlemerchandisestore.com,referral,UNKNOWN,1,0,8,3,1,20618.000000,0
1,8.897099e+07,1612034970969348,20210130,desktop,0,Web,Chrome,United States,Salt Lake City,google,organic,UNKNOWN,1,0,4,2,1,26.000000,0
2,5.210404e+07,1611968241031361,20210130,tablet,0,Web,Safari,United States,(not set),google,organic,UNKNOWN,1,0,4,2,1,12.000000,0
3,7.754877e+07,1612040704762043,20210130,desktop,0,Macintosh,Chrome,United States,New Orleans,(direct),(none),UNKNOWN,1,0,6,2,1,5019.000000,0
4,6.822834e+06,1611998301676244,20210130,mobile,1,Web,Safari,Canada,Brampton,(direct),(none),UNKNOWN,1,0,7,2,2,82.000000,0
5,3.303965e+06,1611983150703412,20210130,mobile,1,iOS,Safari,United States,Los Angeles,<Other>,referral,UNKNOWN,1,0,4,2,1,24.000000,0
6,5.390972e+07,1611972983188588,20210130,desktop,0,Web,Chrome,Cyprus,(not set),(direct),(none),UNKNOWN,1,0,4,2,1,44.000000,0
7,5.997183e+06,1612004659459301,20210130,mobile,1,Web,Chrome,United States,(not set),<Other>,referral,UNKNOWN,0,0,3,2,1,48837.411765,0
8,3.279238e+07,1612036389028482,20210130,desktop,0,Windows,Chrome,Turkey,Adana,<Other>,referral,UNKNOWN,0,0,24,6,3,79106.000000,0
9,7.534357e+06,1612050034319156,20210130,desktop,0,Web,Chrome,Canada,Calgary,(direct),(none),UNKNOWN,0,1,4,2,1,147.000000,0


---

In [181]:
df = pd.read_csv('../data/04_feature/df_fe_train.csv')

In [182]:
df

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_desktop,c_device_category_mobile,c_is_mobile_0,c_is_mobile_1,c_operating_system_Web,c_operating_system_Macintosh,c_operating_system_iOS,...,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart,c_weekday,c_visit_start_hour
0,1.772899e+07,1.611713e+15,20210127,1,0,0,1,1,0,0,...,0,1,1.0,61.0,19.0,1.0,34848.000000,0,1,1
1,3.431204e+07,1.611721e+15,20210127,1,0,0,1,0,1,0,...,0,1,0.0,5.0,2.0,1.0,1125.000000,0,1,2
2,6.149654e+06,1.611737e+15,20210127,0,1,1,0,0,0,1,...,1,0,0.0,318.0,87.0,5.0,479071.000000,0,1,3
3,5.319757e+07,1.611623e+15,20210126,1,0,0,1,1,0,0,...,0,1,0.0,3.0,1.0,1.0,48837.411765,0,2,4
4,3.294631e+07,1.611649e+15,20210126,1,0,0,1,1,0,0,...,1,0,1.0,27.0,9.0,1.0,24954.000000,0,2,3
5,4.367234e+06,1.611740e+15,20210127,0,1,1,0,0,0,0,...,0,1,1.0,6.0,2.0,1.0,1459.000000,0,1,5
6,5.588531e+07,1.611681e+15,20210126,1,0,0,1,1,0,0,...,0,1,0.0,6.0,2.0,1.0,12633.000000,0,2,6
7,5.331924e+07,1.611670e+15,20210126,1,0,0,1,0,1,0,...,1,0,0.0,5.0,2.0,1.0,1420.000000,0,2,7
8,8.550651e+07,1.611768e+15,20210127,1,0,0,1,1,0,0,...,0,1,0.0,4.0,2.0,1.0,32.000000,0,1,6
9,3.268039e+07,1.611741e+15,20210127,1,0,0,1,1,0,0,...,0,1,1.0,7.0,2.0,1.0,11091.000000,0,1,5


In [206]:
features_to_exclude = ['c_is_mobile', 'c_operating_system', 'c_weekday', 'n_total_time_on_site']
# features_to_exclude = []

In [207]:
df_cp = df.copy()

In [208]:
for feature in features_to_exclude:
    feature_list = [item for item in df.columns if re.compile(f'^{feature}').match(item)]
    df_cp = df_cp.drop(feature_list, axis=1)

In [209]:
df_cp

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category_desktop,c_device_category_mobile,c_browser_Chrome,c_browser_Safari,c_browser_Edge,c_browser_<Other>,c_country,...,c_traffic_medium_(data deleted),c_traffic_campaign,c_is_first_visit_0,c_is_first_visit_1,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,y_added_to_cart,c_visit_start_hour
0,1.772899e+07,1.611713e+15,20210127,1,0,1,0,0,0,1,...,0,1,0,1,1.0,61.0,19.0,1.0,0,1
1,3.431204e+07,1.611721e+15,20210127,1,0,1,0,0,0,2,...,0,1,0,1,0.0,5.0,2.0,1.0,0,2
2,6.149654e+06,1.611737e+15,20210127,0,1,0,1,0,0,3,...,0,1,1,0,0.0,318.0,87.0,5.0,0,3
3,5.319757e+07,1.611623e+15,20210126,1,0,0,0,1,0,4,...,0,1,0,1,0.0,3.0,1.0,1.0,0,4
4,3.294631e+07,1.611649e+15,20210126,1,0,1,0,0,0,2,...,0,1,1,0,1.0,27.0,9.0,1.0,0,3
5,4.367234e+06,1.611740e+15,20210127,0,1,1,0,0,0,5,...,0,1,0,1,1.0,6.0,2.0,1.0,0,5
6,5.588531e+07,1.611681e+15,20210126,1,0,1,0,0,0,1,...,0,1,0,1,0.0,6.0,2.0,1.0,0,6
7,5.331924e+07,1.611670e+15,20210126,1,0,1,0,0,0,6,...,0,1,1,0,0.0,5.0,2.0,1.0,0,7
8,8.550651e+07,1.611768e+15,20210127,1,0,0,0,1,0,7,...,0,1,0,1,0.0,4.0,2.0,1.0,0,6
9,3.268039e+07,1.611741e+15,20210127,1,0,1,0,0,0,1,...,0,1,0,1,1.0,7.0,2.0,1.0,0,5
